# Lecture 05 Deep Learning with Keras

## Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
LECTURE_ID = "05"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", LECTURE_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Building an Image Classifier

First let's import TensorFlow and Keras.

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
tf.__version__

'2.1.0'

In [4]:
keras.__version__

'2.2.4-tf'

# Exercise 

*Exercise: Train a deep MLP on the MNIST dataset (you can load it using `keras.datasets.mnist.load_data()`. See if you can get over 98% precision. Try adding all the bells and whistles—save checkpoints, use early stopping, and plot learning curves using TensorBoard.*

Steps to carry out:
- load the dataset 
- explore dataset and normalise it
- define the NN model
- compile the model
- evaluate the model


## Load the dataset

In [5]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 6s 1us/step


In [6]:
X_train_full.shape
y_train_full.shape

(60000,)

## Validation set & Normalisation 

In [7]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

## Model

In [8]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
# keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")
model.add(keras.layers.Dense(300, activation="relu",kernel_initializer="he_normal"))
model.add(keras.layers.Dense(100, activation="relu",kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [10]:
# to plot

## Compile

In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=["accuracy"])

In [12]:
run_index = 1
run_logdir = os.path.join(os.curdir, 'myminst_logs','run_{:03d}'.format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 12s 221us/sample - loss: 0.5830 - accuracy: 0.8439 - val_loss: 0.2978 - val_accuracy: 0.9174
Epoch 2/30
55000/55000 [==============================] - 11s 192us/sample - loss: 0.2820 - accuracy: 0.9183 - val_loss: 0.2357 - val_accuracy: 0.9342
Epoch 3/30
55000/55000 [==============================] - 9s 172us/sample - loss: 0.2307 - accuracy: 0.9346 - val_loss: 0.2018 - val_accuracy: 0.9446
Epoch 4/30
55000/55000 [==============================] - 9s 170us/sample - loss: 0.1972 - accuracy: 0.9443 - val_loss: 0.1791 - val_accuracy: 0.9490
Epoch 5/30
55000/55000 [==============================] - 10s 185us/sample - loss: 0.1733 - accuracy: 0.9507 - val_loss: 0.1554 - val_accuracy: 0.9572
Epoch 6/30
55000/55000 [==============================] - 10s 185us/sample - loss: 0.1544 - accuracy: 0.9566 - val_loss: 0.1443 - val_accuracy: 0.9586
Epoch 7/30
55000/55000 [=======================

In [13]:
# %load_ext tensorboard
# %tensorboard --logdir=./myminst_logs --port=6006

In [14]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 91us/sample - loss: 0.0746 - accuracy: 0.9773


[0.07457246828032657, 0.9773]